In [6]:
def readin(filename):
    with open(filename) as f:
        lines = f.readlines()

    #something to remove \n 
    str_lines = []
    for l in lines:
        if l[-1:]=="\n":
            str_lines.append(l[0:-1])
        else: 
            str_lines.append(l)

    #check all was fine
    print(lines[-2:])
    print(str_lines[-2:])
    return str_lines

#elf_data = readin("Day15input.txt")

# --- Day 15: Beacon Exclusion Zone ---

You feel the ground rumble again as the distress signal leads you to a large network of subterranean tunnels. You don't have time to search them all, but you don't need to: your pack contains a set of deployable sensors that you imagine were originally built to locate lost Elves.

The sensors aren't very powerful, but that's okay; your handheld device indicates that you're close enough to the source of the distress signal to use them. You pull the emergency sensor system out of your pack, hit the big button on top, and the sensors zoom off down the tunnels.

Once a sensor finds a spot it thinks will give it a good reading, it attaches itself to a hard surface and begins monitoring for the nearest signal source beacon. Sensors and beacons always exist at integer coordinates. Each sensor knows its own position and can determine the position of a beacon precisely; however, sensors can only lock on to the one beacon closest to the sensor as measured by the Manhattan distance. (There is never a tie where two beacons are the same distance to a sensor.)

It doesn't take long for the sensors to report back their positions and closest beacons (your puzzle input). For example:


In [7]:
test_data = ["Sensor at x=2, y=18: closest beacon is at x=-2, y=15",
"Sensor at x=9, y=16: closest beacon is at x=10, y=16",
"Sensor at x=13, y=2: closest beacon is at x=15, y=3",
"Sensor at x=12, y=14: closest beacon is at x=10, y=16",
"Sensor at x=10, y=20: closest beacon is at x=10, y=16",
"Sensor at x=14, y=17: closest beacon is at x=10, y=16",
"Sensor at x=8, y=7: closest beacon is at x=2, y=10",
"Sensor at x=2, y=0: closest beacon is at x=2, y=10",
"Sensor at x=0, y=11: closest beacon is at x=2, y=10",
"Sensor at x=20, y=14: closest beacon is at x=25, y=17",
"Sensor at x=17, y=20: closest beacon is at x=21, y=22",
"Sensor at x=16, y=7: closest beacon is at x=15, y=3",
"Sensor at x=14, y=3: closest beacon is at x=15, y=3",
"Sensor at x=20, y=1: closest beacon is at x=15, y=3"]

In [8]:
def parse_data(data):
    returnData = []
    for d in data:
        print(d)
        pair = d.split(":")
        sensor = pair[0].split(",")
        d1 = eval(sensor[0].removeprefix("Sensor at x="))
        d2 = eval(sensor[1].removeprefix(" y="))
        beacon = pair[1].split(",")
        d3 = eval(beacon[0].removeprefix(" closest beacon is at x="))
        d4 = eval(beacon[1].removeprefix(" y="))
        returnData.append([d1, d2, d3, d4])
        print(d1, d2, d3, d4)
    return returnData

parsed_test_data = parse_data(test_data)



Sensor at x=2, y=18: closest beacon is at x=-2, y=15
2 18 -2 15
Sensor at x=9, y=16: closest beacon is at x=10, y=16
9 16 10 16
Sensor at x=13, y=2: closest beacon is at x=15, y=3
13 2 15 3
Sensor at x=12, y=14: closest beacon is at x=10, y=16
12 14 10 16
Sensor at x=10, y=20: closest beacon is at x=10, y=16
10 20 10 16
Sensor at x=14, y=17: closest beacon is at x=10, y=16
14 17 10 16
Sensor at x=8, y=7: closest beacon is at x=2, y=10
8 7 2 10
Sensor at x=2, y=0: closest beacon is at x=2, y=10
2 0 2 10
Sensor at x=0, y=11: closest beacon is at x=2, y=10
0 11 2 10
Sensor at x=20, y=14: closest beacon is at x=25, y=17
20 14 25 17
Sensor at x=17, y=20: closest beacon is at x=21, y=22
17 20 21 22
Sensor at x=16, y=7: closest beacon is at x=15, y=3
16 7 15 3
Sensor at x=14, y=3: closest beacon is at x=15, y=3
14 3 15 3
Sensor at x=20, y=1: closest beacon is at x=15, y=3
20 1 15 3



So, consider the sensor at 2,18; the closest beacon to it is at -2,15. For the sensor at 9,16, the closest beacon to it is at 10,16.

Drawing sensors as S and beacons as B, the above arrangement of sensors and beacons looks like this:

                1    1    2    2
        0    5    0    5    0    5
    0 ....S.......................
    1 ......................S.....
    2 ...............S............
    3 ................SB..........
    4 ............................
    5 ............................
    6 ............................
    7 ..........S.......S.........
    8 ............................
    9 ............................
    10 ....B.......................
    11 ..S.........................
    12 ............................
    13 ............................
    14 ..............S.......S.....
    15 B...........................
    16 ...........SB...............
    17 ................S..........B
    18 ....S.......................
    19 ............................
    20 ............S......S........
    21 ............................
    22 .......................B....


In [9]:
def get_grid_dim(pd):
    min_x = pd[0][0]
    max_x = pd[0][0]
    min_y = pd[0][1]
    max_y = pd[0][1]
    for j in pd:
        # print(i)
        # print(j)
        # print(j[0])
        # print(j[1])
        min_y = min(j[1], j[3], min_y)
        max_y = max(j[1], j[3], max_y)
        min_x = min(j[0], j[2], min_x)
        max_x = max(j[0], j[2], max_x)

    return (min_x, max_x, min_y, max_y)

(min_x, max_x, min_y, max_y) = get_grid_dim(parsed_test_data)

def init_grid(min_x, max_x, min_y, max_y):
    grid=[]
    for y in range(min_y,max_y+1):
        new_line = []
        for x in range(min_x, max_x+1):    
            new_line.append(".")
        grid.append(new_line)
    return grid

def printout(st):
    for s in st:
        print(' '.join([str(c) for c in s])
)


# grid = init_grid(min_x, max_x, min_y, max_y)
# printout(grid)

# grid[1][2] = 2   #[y,x]
# print(grid)

In [10]:
def lookup(data, min_x, x, min_y, y):
    #print("LOOKUP ", min_x, x, y, len(data[0]), len(data))
    return data[y-min_y][(x-min_x)]

#print(lookup(slice_test, min_x, 500, 0))

def edit(data, min_x, x, min_y, y, new):
    data[y-min_y][(x-min_x)] = new
    return data

In [11]:
def plot_S_B(data, grid, min_x, min_y):
    for d in data:
        edit(grid, min_x, d[0], min_y, d[1], "S")
        edit(grid, min_x, d[2], min_y, d[3], "B")
    return grid

# printout(plot_S_B(parsed_test_data, grid, min_x, min_y))


This isn't necessarily a comprehensive map of all beacons in the area, though. Because each sensor only identifies its closest beacon, if a sensor detects a beacon, you know there are no other beacons that close or closer to that sensor. There could still be beacons that just happen to not be the closest beacon to any sensor. Consider the sensor at 8,7:

                1    1    2    2
        0    5    0    5    0    5
    -2 ..........#.................
    -1 .........###................
    0 ....S...#####...............
    1 .......#######........S.....
    2 ......#########S............
    3 .....###########SB..........
    4 ....#############...........
    5 ...###############..........
    6 ..#################.........
    7 .#########S#######S#........
    8 ..#################.........
    9 ...###############..........
    10 ....B############...........
    11 ..S..###########............
    12 ......#########.............
    13 .......#######..............
    14 ........#####.S.......S.....
    15 B........###................
    16 ..........#SB...............
    17 ................S..........B
    18 ....S.......................
    19 ............................
    20 ............S......S........
    21 ............................
    22 .......................B....

This sensor's closest beacon is at 2,10, and so you know there are no beacons that close or closer (in any positions marked #).


In [12]:
no_beacon = set()

grid = init_grid(min_x, max_x, min_y, max_y)
grid = plot_S_B(parsed_test_data, grid, min_x, min_y)
#printout(grid)

def rule_out_positions(sen, min_x, max_x, min_y, max_y, no_beacon):
    manhattan = abs(sen[0]-sen[2])+ abs(sen[1]-sen[3])
    # current_row - rule out manhattan left and right
    for yrange in range((manhattan*-1), manhattan):
        y = sen[1]+yrange
        for x in range(sen[0]-(manhattan-yrange), sen[0]+(manhattan-yrange)+1):
            if x>= min_x and x <= max_x and y >= min_y and y <= max_y:
                if x!= sen[2] or y!=sen[3]:
                    no_beacon.add((x, y))
    # y = sen[1]+1
    # for x in range(sen[0]-(manhattan-1), sen[0]+manhattan-1+1):
    #     if x>= min_x and x <= max_x and y >= min_y and y <= max_y:
    #         no_beacon.add((x, y))
    
    #row above - rule out manhattan -1 left and right

    return no_beacon
    
for d in parsed_test_data:
    no_beacon = rule_out_positions(d, min_x, max_x, min_y, max_y, no_beacon)


In [13]:


#print(no_beacon)

def add_to_grid(no_beacon, grid, min_x, min_y):
    for n in no_beacon:
        if lookup(grid, min_x, n[0], min_y, n[1]) == ".":
            edit(grid, min_x, n[0], min_y, n[1], "#")
    return grid

grid = add_to_grid(no_beacon, grid, min_x, min_y)

printout(grid)

# # # # S # # # # # # # # # # # # # # # # # # # # # # #
# # # # # # # # # # # # # # # # # # # # # # S # # # # #
# # # # # # # # # # # # # # # S # # # # # # # # # # # #
# # # # # # # # # # # # # # # # S B # # # # # # # # # #
# # # # # # # # # # # # # # # # # # # # # # # # # # # .
# # # # # # # # # # # # # # # # # # # # # # # # # # . .
# # # # # # # # # # # # # # # # # # # # # # # # # # # #
. # # # # # # # # # S # # # # # # # S # # # # # # # # #
# # # # # # # # # # # # # # # # # # # # # # # # # # # #
# # # # # # # # # # # # # # # # # # # # # # # # # # # #
# # # # B # # # # # # # # # # # # # # # # # # # # # # #
# # S # # # # # # # # # # # # # # # # # # # # # # # # #
# # # # # # # # # # # # # # # # # # # # # # # # # # # #
# # # # # # # # # # # # # # # # # # # # # # # # # # # #
# # # # # # # # # # # # # # S # # # # # # # S # # # # #
B # # # # # # # # # # # # # # # # # # # # # # # # # # #
# # # # # # # # # # # S B # # # # # # # # # # # # # # #
# # # # # # # # # # # # # # # # S # # # # # # # 


None of the detected beacons seem to be producing the distress signal, so you'll need to work out where the distress beacon is by working out where it isn't. For now, keep things simple by counting the positions where a beacon cannot possibly be along just a single row.

So, suppose you have an arrangement of beacons and sensors like in the example above and, just in the row where y=10, you'd like to count the number of positions a beacon cannot possibly exist. The coverage from all sensors near that row looks like this:

                    1    1    2    2
        0    5    0    5    0    5
    9 ...#########################...
    10 ..####B######################..
    11 .###S#############.###########.


In [14]:
def get_no_beacons_in_row(grid, row, min_x):
    sum = 0
    for g in grid[row+min_x]:
        if g=="#" or g=="S":
            sum = sum+1
    return sum

get_no_beacons_in_row(grid, 10, min_x)

28


In this example, in the row where y=10, there are 26 positions where a beacon cannot be present.

Consult the report from the sensors you just deployed. In the row where y=2000000, how many positions cannot contain a beacon?